In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
diamantes = pd.read_csv("DiamondsPrices2022.csv")

In [3]:
diamantes.drop(['Unnamed: 0'], axis=1, inplace=True)

In [4]:
diamantes = diamantes.replace({'clarity': {"IF":7, "VVS1": 6, "VVS2": 5, "VS1": 4, "VS2": 3, "SI1": 2, "SI2": 1, "I1": 0}})

C:\Users\shirl\AppData\Local\Temp\ipykernel_21876\477035910.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  diamantes = diamantes.replace({'clarity': {"IF":7, "VVS1": 6, "VVS2": 5, "VS1": 4, "VS2": 3, "SI1": 2, "SI2": 1, "I1": 0}})


In [5]:
diamantes = diamantes.replace({'cut': {"Fair":0, "Good": 1, "Very Good": 2, "Premium": 3, "Ideal": 4}})

C:\Users\shirl\AppData\Local\Temp\ipykernel_21876\1024795902.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  diamantes = diamantes.replace({'cut': {"Fair":0, "Good": 1, "Very Good": 2, "Premium": 3, "Ideal": 4}})


In [6]:
diamantes = diamantes.replace({'color': {"D": 6, "E": 5, "F": 4, "G": 3, "H": 2, "I": 1, "J":0}})

C:\Users\shirl\AppData\Local\Temp\ipykernel_21876\2452364888.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  diamantes = diamantes.replace({'color': {"D": 6, "E": 5, "F": 4, "G": 3, "H": 2, "I": 1, "J":0}})


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [8]:
X = diamantes.drop(columns=["price"])
y = diamantes["price"]

n_bins = 5  
bin_edges = np.linspace(y.min(), y.max(), n_bins + 1)
y_binned = np.digitize(y, bins=bin_edges[1:-1])

X_train, X_test, y_train, y_test = train_test_split(X, y_binned, test_size=0.2, random_state=1889)

modelo = RandomForestClassifier(n_estimators=500, max_depth=4, min_samples_leaf=20, max_features=5, random_state=1889)
modelo.fit(X_train, y_train)

RandomForestClassifier(max_depth=4, max_features=5, min_samples_leaf=20,
                       n_estimators=500, random_state=1889)

In [9]:
predic_train = modelo.predict(X_train)

In [10]:
predic_test = modelo.predict(X_test)

In [11]:
df_predic_test = pd.DataFrame(predic_test)

In [12]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train, modelo.predict(X_train))

0.8771376929137508

In [13]:
#import pickle
#with open('ML_entrenado.pkl', 'wb') as file:
#    pickle.dump(modelo, file)
#
#with open('ML_entrenado.pkl', 'rb') as file:
#    loaded_model = pickle.load(file)
#
#predictions = loaded_model.predict(X_test)
#print(predictions)

In [14]:
df = pd.read_csv("DiamondsPrices2022.csv")
columns = df.drop('price', axis=1).columns 

In [15]:
juntos = [df, X_train, X_test, df_predic_test]

juntos = {
    "df": df, 
    "X_train": X_train, 
    "X_test": X_test, 
    "df_predic_test": df_predic_test}


### **API**

In [16]:
from flask import Flask, request, jsonify
import sqlite3
from operator import index
from altair import Orient
from matplotlib.font_manager import json_dump

In [17]:
df_todos = {}

for nombre, dtf in juntos.items():
    df_todos[nombre] = dtf.to_dict(orient="records")

In [18]:
with open("C:/Users/shirl/Desktop/copia_que_puedo_tocar/Proyecto_API/df_final.json", "w") as json_file:
    json.dump(df_todos, json_file, indent=6)

In [19]:
app = Flask(__name__)
app.config['DEBUG'] = False

In [20]:
#with open('ML_entrenado.pkl', 'rb') as file:
#    model = pickle.load(file)
#    
#df = pd.read_csv("DiamondsPrices2022.csv")
#columns = df.drop('price', axis=1).columns 

In [21]:
try:
    with open("C:/Users/shirl/Desktop/copia_que_puedo_tocar/Proyecto_API/df_final.json", "r") as json_file:
        df_final = json.load(json_file)
except Exception as e:
        df_final = None
        print(f"Error loading JSON file: {e}")

In [22]:
X_test

,carat,cut,color,clarity,depth,table,x,y,z
47195,0.59,2,5,3,59.9,56.0,5.46,5.49,3.28
6724,0.27,4,4,3,60.3,56.0,4.24,4.27,2.56
21395,1.24,3,4,3,62.4,60.0,6.83,6.80,4.25
43930,0.30,4,1,5,62.2,56.0,4.27,4.31,2.67
43260,0.30,2,0,4,63.4,57.0,4.26,4.23,2.69
...,...,...,...,...,...,...,...,...,...
51865,0.52,4,6,5,60.4,57.0,5.25,5.28,3.18
19540,1.42,4,1,3,60.4,55.0,7.28,7.38,4.43
19697,0.36,2,1,2,63.1,57.0,4.55,4.51,2.86
6063,1.01,2,3,1,61.8,58.0,6.36,6.39,3.94


In [23]:
@app.route('/', methods=['GET'])
def home():
    return "<h1>Diamond Prices Prediction API</h1><p>This site is a prototype API for predicting diamond prices.</p>"

@app.route('/diamantes/all', methods=['GET'])
def api_all():
    return jsonify(df_final["df"])

@app.route('/X_train', methods=['GET'])
def api_X_train():
    return jsonify(df_final["X_train"])

@app.route('/X_test', methods=['GET'])
def api_X_test():
    return jsonify(df_final["X_test"])

@app.route('/predicciones', methods=['GET'])
def api_predicciones():
    return jsonify(df_final["df_predic_test"])

@app.route('/diamantes/by_color', methods=['GET'])
def by_color():
    if "color" in request.args:
        color = str(request.args["color"])
    else:
        return 'Error, agrega un color, ej: colorless: F, E, D. near colorless: J, I, H, G'

    df_filtrado = df[df["color"]==color]
    return jsonify(df_filtrado.to_dict(orient="records"))

@app.route('/diamantes/by_cuts', methods=['GET'])
def by_cut():
    if "cut" in request.args:
        cut = str(request.args["cut"])
    else:
        return 'Error, agrega un corte, orden ascendente: Fair, Good, Very Good, Premium, Ideal'

    df_filt = df[df["cut"]==cut]
    return jsonify(df_filt.to_dict(orient="records"))

if __name__ == '__main__':
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [04/Aug/2024 11:14:28] "GET /diamantes/by_cuts?cut=Ideal HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2024 11:14:28] "GET /diamantes/by_color?color=E HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2024 11:14:29] "GET /diamantes/all HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2024 11:14:29] "GET /X_train HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2024 11:14:30] "GET /X_test HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2024 11:14:30] "GET /predicciones HTTP/1.1" 200 -
